# Final Project Assignment
# Yihnew Eshetu (yte9pc), Nathan England (nle4bz), and Karyne Williams (kw7me)

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import *
from pyspark.ml import Pipeline 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics 

import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#import plotly_express as px

#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName('Paid Classifier')
#     .config(“spark.executor.memory”, ‘15g’)
#     .config(‘spark.executor.cores’, ‘1’)
#     .config(‘spark.executor.instances’, ‘1’)
     .config('spark.driver.memory','15g')
    .getOrCreate()
)

## Data Import and Preprocessing

### Function Reads Taxi Zone

In [2]:
def taxiZone():
    taxi = spark.read.csv("/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Taxi_Zone/taxi_zone_lookup.csv", header = True)
    return taxi

### Function Reads Yellow Cab CSVs

In [3]:
def yellowCSVToDF():
    location = "/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Data/Raw_Data/"
    

    files = sorted(glob.glob(location + '*.csv'))

    for idx,f in enumerate(files):
        df = spark.read.csv(f, header = True)
        print(f)

        if idx == 0:
            data = df
        else:
            data = data.union(df)
    return data

### Adds Taxi Zone for Pickup and Dropoff

In [4]:
def yellowTaxiZone():
    # Drop Temp View
    spark.catalog.dropTempView("taxiZone")
    spark.catalog.dropTempView("yellowCab")
    
    taxiZone().createTempView("taxiZone")
    yellowCSVToDF().createTempView("yellowCab")
    yellowCab = spark.sql("SELECT yellowCab.*,\
                     CASE WHEN PU.Borough = 'Bronx'\
                          THEN 1\
                          WHEN PU.Borough = 'Brooklyn'\
                          THEN 2\
                          WHEN PU.Borough = 'EWR'\
                          THEN 3\
                          WHEN PU.Borough = 'Manhattan'\
                          THEN 4\
                          WHEN PU.Borough = 'Queens'\
                          THEN 5\
                          WHEN PU.Borough = 'Staten Island'\
                          THEN 6\
                          ELSE 7\
                     END as PUBorough,\
                     PU.Zone as PUZone,\
                     CASE WHEN PU.service_zone = 'Airports'\
                          THEN 1\
                          WHEN PU.service_zone = 'Boro Zone'\
                          THEN 2\
                          WHEN PU.service_zone = 'EWR'\
                          THEN 3\
                          WHEN PU.service_zone = 'Yellow Zone'\
                          THEN 4\
                          ELSE 5\
                     END as PUService_Zone,\
                     CASE WHEN DO.Borough = 'Bronx'\
                          THEN 1\
                          WHEN DO.Borough = 'Brooklyn'\
                          THEN 2\
                          WHEN DO.Borough = 'EWR'\
                          THEN 3\
                          WHEN DO.Borough = 'Manhattan'\
                          THEN 4\
                          WHEN DO.Borough = 'Queens'\
                          THEN 5\
                          WHEN DO.Borough = 'Staten Island'\
                          THEN 6\
                          ELSE 7\
                     END as DOBorough,\
                     DO.Zone as DOZone,\
                     CASE WHEN DO.service_zone = 'Airports'\
                          THEN 1\
                          WHEN DO.service_zone = 'Boro Zone'\
                          THEN 2\
                          WHEN DO.service_zone = 'EWR'\
                          THEN 3\
                          WHEN DO.service_zone = 'Yellow Zone'\
                          THEN 4\
                          ELSE 5\
                     END as DOService_Zone\
                     FROM yellowCab \
                     INNER JOIN taxiZone PU\
                            ON yellowCab.PULocationID = PU.LocationID \
                     INNER JOIN taxiZone DO\
                            ON yellowCab.DOLocationID = DO.LocationID")
    return yellowCab
data = yellowTaxiZone()

/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Data/Raw_Data/yellow_tripdata_2019-05.csv
/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Data/Raw_Data/yellow_tripdata_2019-06.csv
/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Data/Raw_Data/yellow_tripdata_2019-07.csv
/sfs/qumulo/qhome/yte9pc/Project/Yellow_Cab_Data/Data/Raw_Data/yellow_tripdata_2019-08.csv


### Drop duplicates

In [5]:
data = data.dropDuplicates()

### Defining Column Types

In [6]:
data = data.withColumn("VendorID", data["VendorID"].cast(IntegerType()))\
    .withColumn("tpep_pickup_datetime", data["tpep_pickup_datetime"].cast(TimestampType()))\
    .withColumn("tpep_dropoff_datetime", data["tpep_dropoff_datetime"].cast(TimestampType()))\
    .withColumn("passenger_count", data["passenger_count"].cast(IntegerType()))\
    .withColumn("trip_distance", data["trip_distance"].cast(FloatType()))\
    .withColumn("RatecodeID", data["RatecodeID"].cast(IntegerType()))\
    .withColumn("PULocationID", data["PULocationID"].cast(IntegerType()))\
    .withColumn("DOLocationID", data["DOLocationID"].cast(IntegerType()))\
    .withColumn("payment_type", data["payment_type"].cast(IntegerType()))\
    .withColumn("fare_amount", data["fare_amount"].cast(FloatType()))\
    .withColumn("extra", data["extra"].cast(FloatType()))\
    .withColumn("mta_tax", data["mta_tax"].cast(FloatType()))\
    .withColumn("tip_amount", data["tip_amount"].cast(FloatType()))\
    .withColumn("tolls_amount", data["tolls_amount"].cast(FloatType()))\
    .withColumn("improvement_surcharge", data["improvement_surcharge"].cast(FloatType()))\
    .withColumn("total_amount", data["total_amount"].cast(FloatType()))\
    .withColumn("congestion_surcharge", data["congestion_surcharge"].cast(FloatType()))

### Adding Additional Features

In [7]:
data = data.withColumn("pickup_week_day", date_format("tpep_pickup_datetime", "u").cast(IntegerType()))\
    .withColumn("pickup_hour", hour("tpep_pickup_datetime"))\
    .withColumn("dropoff_week_day", date_format("tpep_dropoff_datetime", "u").cast(IntegerType()))\
    .withColumn("dropoff_hour", hour("tpep_dropoff_datetime"))\
    .withColumn("trip_time(mins)", (col("tpep_dropoff_datetime").cast(LongType()) - col("tpep_pickup_datetime").cast(LongType()))/60)\
    .withColumn("distance/time", (col("trip_distance")/col("trip_time(mins)")))
data.select(["tpep_pickup_datetime", "pickup_hour", "pickup_week_day", "tpep_dropoff_datetime", 
             "dropoff_week_day", "dropoff_hour", "trip_time(mins)"]).show(3)

+--------------------+-----------+---------------+---------------------+----------------+------------+-----------------+
|tpep_pickup_datetime|pickup_hour|pickup_week_day|tpep_dropoff_datetime|dropoff_week_day|dropoff_hour|  trip_time(mins)|
+--------------------+-----------+---------------+---------------------+----------------+------------+-----------------+
| 2019-05-01 00:44:57|          0|              3|  2019-05-01 00:50:11|               3|           0|5.233333333333333|
| 2019-05-01 00:59:57|          0|              3|  2019-05-01 01:13:18|               3|           1|            13.35|
| 2019-05-01 00:09:43|          0|              3|  2019-05-01 00:14:36|               3|           0|4.883333333333334|
+--------------------+-----------+---------------+---------------------+----------------+------------+-----------------+
only showing top 3 rows



### Drop na

In [8]:
data = data.dropna()

### Print Schema

In [9]:
data.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- PUBorough: integer (nullable = false)
 |-- PUZone: string (nullable = true)
 |-- PUService_Zone: integer (nullable = false)
 |-- DOBorough: integer (nullable = false)
 |-- 

### Remove cash and credit card payment where total amount is less than 0

In [10]:
data = data.filter(((data.total_amount >= 0) & (data.payment_type <= 2)) | (data.payment_type > 2))

### Remove rides with trip time less than 0 mins

In [11]:
data = data.filter((col('trip_time(mins)') >= 0) & (col('trip_distance') >= 0))

## We will construct a model to determine whether or not a passenger will pay. 
What factors determine whether or not a passenger will pay? 

### Logistic Regression Model Specific Preprocessing

### Remove Unknown Payment Types and Voided Trips

In [12]:
#Payment type: 5 = Unknown, 6 = Voided Trip
data = data.filter(col("payment_type") < 5)

### Create label to identify whether or not a passenger paid

In [13]:
data = data.withColumn('paid', F.when(col("payment_type") <= 2, 1).otherwise(0))

In [14]:
data.groupBy('paid').count().show()

+----+--------+
|paid|   count|
+----+--------+
|   1|17743064|
|   0|  141077|
+----+--------+



### Balance Data by Upsampling Negatives and Downsampling Positives

In [15]:
negatives = data.filter(data['paid'] == 0)
positives = data.filter(data['paid'] == 1)

#Downsample positives to 1% of orignal data size
pos_undersampled = positives.sample(withReplacement = False, fraction = .01, seed = 123)

neg_count = negatives.count()
pos_count = pos_undersampled.count() 
ratio = pos_count/neg_count

neg_oversampled = negatives.sample(withReplacement=True, fraction=ratio, seed = 123 )
data = pos_undersampled.unionAll(neg_oversampled)

In [16]:
data.groupBy('paid').count().show()

+----+------+
|paid| count|
+----+------+
|   1|176927|
|   0|176830|
+----+------+



### **Bench Mark Model**

### Select variables

In [17]:
vars_to_keep = ['trip_distance', 'fare_amount']
bm_data = data.select(col('paid').alias('label'), *vars_to_keep)
bm_data.printSchema()

root
 |-- label: integer (nullable = false)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)



### Data Splitting: Training and Test Sets

In [18]:
(bm_train, bm_test) = bm_data.randomSplit([.8, .2], seed = 123)

### Pipeline

In [19]:
va = VectorAssembler(inputCols= vars_to_keep, outputCol="assembled_features")
scale = StandardScaler(inputCol= va.getOutputCol(), outputCol="features")
lr = LogisticRegression(featuresCol = scale.getOutputCol(), labelCol = 'label', maxIter=10)

# Build the pipeline
bm_stages = [va, scale, lr]
bm_pipeline = Pipeline(stages= bm_stages)

### Cross Validation

In [20]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator= bm_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

# Run cross-validation, and choose the best set of parameters.
bm_cvModel = crossval.fit(bm_train)

# Make a prediction
bm_prediction = bm_cvModel.transform(bm_test).select('label', 'prediction')

In [21]:
bm_prediction.printSchema()

root
 |-- label: integer (nullable = false)
 |-- prediction: double (nullable = false)



### Model Evaluation

In [22]:
bm_preds_and_labels = bm_prediction.select(['prediction','label']).withColumn('label', bm_prediction['label'].cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
bm_preds_and_labels = bm_preds_and_labels.select(['prediction','label']).rdd.map(tuple)

In [23]:
bm_multi_metrics = MulticlassMetrics(bm_preds_and_labels)
bm_bin_metrics = BinaryClassificationMetrics(bm_preds_and_labels)
print("Confusion Matrix")
print(bm_multi_metrics.confusionMatrix().toArray())

Confusion Matrix
[[24856. 10308.]
 [20517. 14815.]]


In [24]:
print("Summary Stats")
print("Accuracy = %s" % bm_multi_metrics.accuracy)
print("Precision = %s" % bm_multi_metrics.precision(1.0))
print("Recall = %s" % bm_multi_metrics.recall(1.0))
print("F1 Score = %s" % bm_multi_metrics.fMeasure(1.0))
print("Area under ROC = %s" % bm_bin_metrics.areaUnderROC)

Summary Stats
Accuracy = 0.5627411484339537
Precision = 0.5896986824821876
Recall = 0.4193082757839918
F1 Score = 0.4901166156645439
Area under ROC = 0.5630837818460399


### **Complex Model**

### Select variables

In [25]:
comp_vars_to_keep = ['passenger_count', 'trip_distance', 'RatecodeID', 'fare_amount', 'tolls_amount', 
                    'improvement_surcharge', 'congestion_surcharge', 'PUBorough', 'pickup_week_day', 'pickup_hour']
comp_data = data.select(col('paid').alias('label'), *comp_vars_to_keep)
comp_data.printSchema()

root
 |-- label: integer (nullable = false)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- PUBorough: integer (nullable = false)
 |-- pickup_week_day: integer (nullable = true)
 |-- pickup_hour: integer (nullable = true)



### Data Splitting: Training and Test Sets

In [26]:
(comp_train, comp_test) = comp_data.randomSplit([.8, .2], seed = 123)

### Pipeline

In [27]:
va = VectorAssembler(inputCols= comp_vars_to_keep, outputCol="assembled_features")
scale = StandardScaler(inputCol= va.getOutputCol(), outputCol="features")
lr = LogisticRegression(featuresCol = scale.getOutputCol(), labelCol = 'label', maxIter=10)

# Build the pipeline
comp_stages = [va, scale, lr]
comp_pipeline = Pipeline(stages= comp_stages)

### Cross Validation

In [28]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator= comp_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

# Run cross-validation, and choose the best set of parameters.
comp_cvModel = crossval.fit(comp_train)

# Make a prediction
comp_prediction = comp_cvModel.transform(comp_test).select('label', 'prediction')

In [29]:
comp_prediction.printSchema()

root
 |-- label: integer (nullable = false)
 |-- prediction: double (nullable = false)



### Model Evaluation

In [30]:
comp_preds_and_labels = comp_prediction.select(['prediction','label']).withColumn('label', comp_prediction['label'].cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
comp_preds_and_labels = comp_preds_and_labels.select(['prediction','label']).rdd.map(tuple)

In [31]:
comp_multi_metrics = MulticlassMetrics(comp_preds_and_labels)
comp_bin_metrics = BinaryClassificationMetrics(comp_preds_and_labels)
print("Confusion Matrix")
print(comp_multi_metrics.confusionMatrix().toArray())

Confusion Matrix
[[14705. 20459.]
 [ 2910. 32422.]]


In [32]:
print("Summary Stats")
print("Accuracy = %s" % comp_multi_metrics.accuracy)
print("Precision = %s" % comp_multi_metrics.precision(1.0))
print("Recall = %s" % comp_multi_metrics.recall(1.0))
print("F1 Score = %s" % comp_multi_metrics.fMeasure(1.0))
print("Area under ROC = %s" % comp_bin_metrics.areaUnderROC)

Summary Stats
Accuracy = 0.6685060145256468
Precision = 0.6131124600518144
Recall = 0.9176384014491112
F1 Score = 0.7350843979912257
Area under ROC = 0.6679108854020668


In [33]:
!jupyter nbconvert --to pdf `pwd`/*.ipynb

[NbConvertApp] Converting notebook /sfs/qumulo/qhome/nle4bz/ds5559/Project/Tree Model.ipynb to pdf
[NbConvertApp] Writing 80141 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 80555 bytes to /sfs/qumulo/qhome/nle4bz/ds5559/Project/Tree Model.pdf
[NbConvertApp] Converting notebook /sfs/qumulo/qhome/nle4bz/ds5559/Project/dimensionaModeling.ipynb to pdf
[NbConvertApp] Writing 77525 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully c